# Linear Regression and Gradient Descent
Up to this point we have investigated a number of different data sets.   Each dataset had a number of data samples, and each sample had a **label** and a number of **features**.  In each case we were either told or we were trying to find a relationship between the features and the labels, with the goal of developing a **model** which would allow us to predict the label, given new **unseen** data.

Some examples:


1.   Housing data: Here we were trying to find a linear relationship between various features in a  housing data set and the median house value across California districts.  This used the sklearn LinearRegression package to perform **regression**.
2.   MNIST digits: Here we used the pixels in a 28x28 image as our features, and digit values as their labels.  We used the sklearn  LinearSVC package to perform **classification**.
3.   Pulsar data:  Here we used descriptive features to classify samples as either pulsars or background.   We used 
the sklearn packages for decision trees and random forests to perform classification.
In all of these cases we have used method from the **sckit learn** package.   

Let's take some time and investigate some of the underlying mathematics for these tools and write our own tools, at least for some simple implementations.   

We will start with Multiple Linear Regression,  (aka multivariable regression) which has  dependent variable and multiple independent variables.  Note that this is not formally Multivariate regression, which has multiple dependent variables and multiple independent variables.



# Generation of a Toy Dataset
Later on we will apply our methods to a previous example where we used sklearn.    But for now, we will generate a toy dataset, with two independent variables, which each have a fixed relationship with a single dependent variable.

In [0]:
import numpy as np

numPoints = 200
X1 = 10.0* np.random.rand(numPoints,1)
X2 = 20.0* np.random.rand(numPoints,1)
beta0 = 4.2
beta1 = -15.2
beta2 = 7.7
#
yrand = np.random.normal(0.0, 10.0, numPoints)
yrand = yrand.reshape(numPoints,1)
y = beta0 + beta1*X1 + beta2*X2
#
# If you want to add some noise to the label, uncomment this line
#y = y + yrand



In [0]:
from tabulate import tabulate


# Visualize the dataset
To see this dataset, we will use Scatter3D from plotly.   By using your mouse you can rotate the plot and verify that the dependent variable **y** is linear in both **X1** and **X2**.

In [0]:
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
  '''))
  init_notebook_mode(connected=False)

In [0]:
#
# Lets plot this
import plotly.plotly as py
import numpy as np
from plotly.offline import iplot
import plotly.graph_objs as go

enable_plotly_in_cell()

trace1 = go.Scatter3d(
    x=np.squeeze(X1),
    y=np.squeeze(X2),
    z=np.squeeze(y),
    mode='markers',
    marker=dict(
        size=5,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.2
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = dict(
    title='Toy Dataset',
    xaxis="X1",
    yaxis="X2",
    zaxis="observed"
)

#iplot(dict(data=data))
#iplot(dict(data=data,layout=layout))

fig = go.Figure(data=data)
iplot(fig,validate=False)


# Linear Regression
We are to imagine that we were given this dataset, with labels **y**, and features **X1** and **X2**, and we are going to dtry to **discover** the underlying linear relationship - if there is one.   Of course since we generated the data we know that there is, but generally we don't know this for sure.   

## The Hypothesis
We will use the following hypothesis: that the labels can be predicted from the features using the following model:
$$h_\theta^{(i)} = y_{pred}^{(i)} = \theta_0  + \theta_1 \cdot X_1^{(i)} +  \theta_2 \cdot X_2^{(i)} $$


The vector of $\theta$ values {$\theta_0,\theta_1,\theta_2$} are the **parameters** of our model.   Our goal is to come up with a procedure to estimate these parameters.

We will also define two counters:
*  *m*: this will run from 1 to the number of samples (or data points) that we have.
*  *n*: this will run from 1 to the number of features that we have

For our current dataset, m=200, and n=2.  Also, note that we have m=200 $h_\theta^{(i)}=y_{pred}^{(i)}$ values.

## Fitting for the parameters of our model
We would like to choose $\theta_0,\theta_1,\theta_2$ so that $h_\theta^{(i)}=y_{pred}^{(i)}$ is a close to all of the $y^{(i)}$ as possible.

To do this, we will minimize the following **cost function** $J(\theta)$:
$$J(\theta) = {1\over{2m}}\sum_{i=i}^m(h_\theta^{(i)}-y^{(i)})^2$$

 $$J(\theta) = {1\over{2m}}\sum_{i=i}^m(\theta_0  + \theta_1 \cdot X_1^{(i)} +  \theta_2 \cdot X_2^{(i)}-y^{(i)})^2$$
 
 ## Gradient Descent
 We want to find the values $\theta_0,\theta_1,\theta_2$  that minimize $J(\theta)$.   The basic idea is to try slowly modify all of our $\theta$ parameters in such a way that we can move in a direction that minimizes J.   Gradient descent is an algorithm that allows us to do this.   The basic idea is this, for each parameter, we iteratively update according to the following scheme:
 $$\theta_j := \theta_j - \alpha {\delta J\over \delta \theta_j}(\theta)$$
 for j=0,1,..n.   Note that we have (n+1) parameters and not $n$ because, although we have $n$ features, due to the intercept term $\theta_0$, we actually have $n+1$ parameters.
 
 The term $\alpha$ is called the **learning rate**, and it controls how quickly we will update (or correct) our parameters to find the minimum of *J(\theta)$.   More on this below.
 
 Note the use of the special symbols ":=".   This means that we need to **simultaneously** update all of the $\theta$ parameters.   
 
 ## Adding a Dummy Features Column
 It turns out that we can make our calculations much easier if we employ a little trick.   For our present problem we have two features, X1 and X2, which take on a variety of different values.  We are going to add, for every sample in our data, a new column X0, which always has the value of 1.0.   By doing this, we can rewrite our cost function like this:
 $$J(\theta) = {1\over{2m}}\sum_{i=i}^m(\theta_0\cdot X_0^{(i)}  + \theta_1 \cdot X_1^{(i)} +  \theta_2 \cdot X_2^{(i)}-y^{(i)})^2$$
 
 We can now write $J(\theta)$  using matrix notation like this:
 $$J(\theta) = {1\over{2m}}\sum_{i=i}^m(X^{(i)} \cdot \theta  -y^{(i)})^2= {1\over{2m}}\sum_{i=i}^m(h_\theta(X^{(i)})  -y^{(i)})^2$$
 
 Let's remember the dimensions of each term:
 1.  $\theta$: this is a matrix of dimension 3x1
 2.  $X$: This is a matrix of dimension mx3 (since we have m samples).
 3. $h_\theta(X)$: This is a matrix of dimension mx1.
 4.  $y$: This is a matrix of dimension mx1.
 
Note that although I described each of the above as *matrices*, we will use numpy (2 dimensional) *arrays* to implement all fo them.
 
 We can implement the above calculation in a single line of code (though we will use 2 for clarity):
 $$hTheta = np.dot(X,Theta)$$
  $$J=np.sum(np.square(hTheta-y))/(2*len(y))$$
  
The actual code is listed here (using slightly different names for the variables):
 

In [0]:
import numpy as np

def calc_cost(Theta,Xp,y):
  hTheta = np.dot(Xp,Theta)
  cost=np.sum(np.square(hTheta-y))/(2*len(y))
  return cost

## Implementing Gradient Descent
In order to implement gradient descent, we first need the gradient - the derivative of J with respect to our $\theta$.

Here is our cost function $J(\theta)$:
 $$J(\theta) = {1\over{2m}}\sum_{i=i}^m(X^{(i)} \cdot \theta  -y^{(i)})^2= {1\over{2m}}\sum_{i=i}^m(h_\theta(X^{(i)})  -y^{(i)})^2$$
 
 And here is the derivative with respect to $\theta$:
 $${\delta J\over \delta \theta_j} = {1\over{m}}\sum_{i=i}^m(X^{(i)} \cdot \theta  -y^{(i)})\cdot X^{(i)}= {1\over{m}}\sum_{i=i}^m(h_\theta(X^{(i)})  -y^{(i)})\cdot X^{(i)}$$
 
 Again - we can implement the above calculation in a single line of code (though we will use 3 for clarity):
  
$hTheta = np.dot(Xp,Theta)$

$delTheta = np.dot(Xp.transpose(),(hTheta-yp))$

$delTheta = delTheta / (2*len(y))$


Notice that we don't have to actually do the summation in the second line - that is taken care of for us when we do the dot-product.

The actual code is listed here (using slightly different names for the variables):

 

In [0]:
def calc_gradient_descent(Theta,Xp,yp):
  hTheta = np.dot(Xp,Theta)
  delTheta = np.dot(Xp.transpose(),(hTheta-yp))
  delTheta /= (2*len(y))
  return delTheta


## Implementing Gradient Descent: The Learning Rate
We mentioned above the $\alpha$ controls the learning rate.   To understand this, refer to this figure:![learningRate](https://i.stack.imgur.com/0tirm.png)

The learing rate $\alpha$ is a parameter you have to set by hand.  Typical values are 0.01-0.0001.   If the learning rate is too small, as shown on the left in the figure, it may take many iterations to get to the minimum of the cost function.   If the learning rate is too high, you may overshoot the minimum and your search might not converge.   The basic way to test this is to plot the cost as a fucntion of the iteration - this will help you see if the learning rate needs to be adjusted.

## Iterating until we converge
The basic algorithm then to implement gradient descent looks like this:
0. Initialize each of the $\theta$ parameters to some reasonable value (0 is common, or a random number).
1. Have an outer loop that checks that we have not exceeded our maximum number of allowed iterations **AND** that are cost is still decreasing.
2. Calculate the gradient and update our parameters like so:
$$\theta_j := \theta_j - \alpha {\delta J\over \delta \theta_j}(\theta)$$
3. Calculate the cost for this iteration and compare it to the cost of the previous iteration.
4. If the change in cost is small enough, declare victory and jump out of the loop.

It is helpful to keep track of the cost for each iteration, so you can plot it and inspect its behavior.   And of course you need to keep track of the last value of the $\theta$ parameters so you can return them.

An implementation of this iteration algorithm is shown below.

In [0]:
import numpy as np

def fit_data(Xp,yp,learningRate,max_iterations,scale=True,delta=0.001):
#
# Get the initial values
  m,features = Xp.shape
  Theta = np.zeros((features,1))
  costList = []
  cost = calc_cost(Theta,Xp,yp)
  cost_change = delta+0.1
  iterations = 0
#
# Iterate until we reach max allowed iterations or the cost function has plateaued
  while (iterations<iterations_max) and (cost_change>delta):
    last_cost = cost
#
# Update the parameters all at once
    Theta = Theta - learningRate*calc_gradient_descent(Theta,Xp,yp)
#
# Calculate the new cost, and see how much it has changed from the previous cost
    cost = calc_cost(Theta,Xp,yp)
    cost_change = last_cost - cost

    costList.append(cost)
    iterations += 1
    #print()
    #print("   iter,cost.cost_change",iterations,cost,cost_change)
    
  return Theta,iterations,costList


## Preparing the data and running the algorithm
We are finally ready to apply our algorithm to our toy dataset.   Remember our dataset has two features X1 and X2, and one label y.   We will need to combine X1 and X2 into a single 2D numpy array, and add a "ones" column (to the front of that array). 

However, before we add the ones column, it is **very helpful** to scale our input features.   This will **greatly** aid in converging.    We will use min-max scaling.   

Keep in mind that if you use scaling:
1.  If you want to predict labels using new (or old for that matter) features, you will have to scale those features using the same scaling parameters.
2.  The parameters we get from minimizing our cost function are using the scaled features, so the $\theta$ values we get won't correspond to the model used to generate the data.  If you want **those** $\theta$ values (and you probably will), you will need to apply a transform to obtain them.   This is shown below.

The code below prepares the data, and then runs the fit.


In [0]:
from sklearn.preprocessing import MinMaxScaler
scl = MinMaxScaler()

#
# Form data
XToFit = np.append(X1,X2,axis=1)
yToFit = y
#print(XToFit.shape,yToFit.shape)
#
# Make sure feature data is normalized
XToFit2 = scl.fit_transform(XToFit)
#XToFit2 = XToFit
#
# Prepend the "ones" column
ones = np.ones((len(XToFit2),1))
XToFit2 = np.append(ones,XToFit2,axis=1)
#
# Make sure label data has the correct shape
yToFit2 = yToFit.reshape(len(yToFit),1)
#
# Check shapes
print("Features shapes: ",XToFit2.shape)
print("Labels shapes:   ",yToFit2.shape)

iterations_max = 50000
learningRate = 0.01
Theta,iterations,costList = fit_data(XToFit2,yToFit2,learningRate,iterations_max,delta=0.00001)
#Theta,costList = fit_data_minimize(XToFit,yToFit,learningRate,iterations)
print("Iterations:",iterations)
print("Final cost:",costList[-1])
print("fit Theta ",Theta)



## Inspection of results.
First off, it the cost looks low, and the number of iterations is well below our maximum.    But the theta values are nothing like our starting values of 
* beta0 = 4.2
* beta1 = -15.2
* beta2 = 7.7

We need to transform them!

The following code assumes a max-min scaler has been applied, and using the parameters from that scaler, recalculates the $\theta$ values.

In [0]:
def coef_transform(Theta,scl):
  mxs = scl.data_max_
  mns = scl.data_min_
  print("pars ",mxs,mns,len(Theta))
  clist = []
  alist = []
  for mn,mx in zip(mns,mxs):
    clist.append(1.0/(mx-mn))
    alist.append(mn/(mx-mn))
  print("clist ",clist)
  print("alist ",alist)
  newTheta = Theta
  print("newTheta shape ",newTheta.shape)
  
  for i in range(len(clist)):
    newTheta[0,0] -= Theta[i+1,0]*alist[i] 
    newTheta[i+1,0] = Theta[i+1,0]*clist[i] 
  return newTheta

#
# Transform coeficients back
Theta_transform = coef_transform(Theta,scl)
print("Theta_transform",Theta_transform)


**Much better!**

## Plotting the cost versus iteration.
Looking at this will ensure that the algorithm converged properly.

In [0]:
enable_plotly_in_cell()
data = go.Scatter(
    x=np.array(range(0,len(costList))),
    y=costList,
    mode='markers',
    name="fitted data"
)


iplot(dict(data=[data]))

# Application to a new dataset
To make sure you understand this, apply the above procedure to the first dataset that we fit: the GDP dataset.

In assignmen3_prep/ch1_scikit_intro.ipynb,  we fit the single feature 'GDP' to the label 'Life satisfaction' using the sklearn LinearRegression model.   Apply the gradient descent method from above to this dataset, and compare the results to our previous results.

The dataset is read in below.



In [0]:
import pandas as pd
url = "https://raw.githubusercontent.com/big-data-analytics-physics/data/master/ch1/gdp_oecd_data_byCountry.csv"
data=pd.read_csv(url)
print(data.head())
